In [ ]:
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier

In [ ]:
file_url = '../dataset/breast-cancer-wisconsin.data'

In [ ]:
col_names = ['Sample code number','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size',
'Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses','Class']

In [ ]:
df = pd.read_csv(file_url, header=None, names=col_names, na_values='?')

In [ ]:
y = df.pop('Class')

In [ ]:
df.drop('Sample code number', axis=1, inplace=True)

In [ ]:
training_rows = int(df.shape[0] * 0.7)
training_rows

In [ ]:
X_train = df[:training_rows]
y_train = y[:training_rows]
X_test = df[training_rows:]
y_test = y[training_rows:]

In [ ]:
X_train.isna().sum()

In [ ]:
num_columns = [col for col in X_train.columns if X_train[col].dtype != 'object']
num_columns

In [ ]:
column_mean = {}
for col in num_columns:
  column_mean[col] = X_train[col].mean()
column_mean

In [ ]:
import pickle
pickle.dump(column_mean, open("columns_mean.pkl", "wb" ) )

In [ ]:
for col in num_columns:
  X_train[col].fillna(column_mean[col], inplace=True)

In [ ]:
rf_model = RandomForestClassifier(random_state=1)
rf_model.fit(X_train, y_train)
joblib.dump(rf_model, "model.pkl") 

In [ ]:
import socket
import threading
import requests
import json
from flask import Flask, jsonify, request
import numpy as np

In [ ]:
app = Flask(__name__)

In [ ]:
trained_model = joblib.load("model.pkl")
var_means = pickle.load(open("columns_mean.pkl", "rb" ) )

In [ ]:
@app.route('/api', methods=['POST'])
def predict():
  data = request.get_json()
  df_test = pd.DataFrame(data, index=[0])
  for col, avg_value in var_means.items():
    df_test[col].fillna(avg_value, inplace=True)
  prediction = trained_model.predict(df_test)
  str_pred = np.array2string(prediction)
  return jsonify(str_pred)

In [ ]:
flask_thread = threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80})
flask_thread.start()

In [ ]:
record = X_test[X_test['Bare Nuclei'].isna()].iloc[0].to_json()
record

In [ ]:
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}
ip_address = socket.gethostbyname(socket.gethostname())

In [ ]:
r = requests.post(f"http://{ip_address}/api", data=record, headers=headers)
r.text